# Classificação de um paciente

## Donwload e Organização

In [4]:
from datetime import datetime
import numpy as np

path = '/mnt/c/Users/davi.nascimento/Documents/siena-eeg/PN00'

f = [i for i in open(f"{path}/Seizures-list-PN00.txt", "r")]


def get_patterns(file):
    patterns = [
        'File name:', 
        'Registration start time:',
        'Registration end time:',
        'Seizure start time:',
        'Seizure end time:'
    ]

    keys = ['name','registration_start','registration_end','seizure_start','seizure_end']
    data = []

    for line in file:
        for idx in range(len(keys)):
            if patterns[idx] in line:
                d = {}
                d[keys[idx]]= line.replace(patterns[idx],'').strip()
                data.append(d)

    return data


def set_intervals(data):
    intervals = [cont for cont in range(len(data)) if 'name' in data[cont].keys()]
    intervals.append(None)
    data_ = []
    
    for cont in range(len(intervals)-1):
        data_.append(data[intervals[cont]:intervals[cont+1]])

    return data_


def dict_update(data):
    d = {}
    for i in data:
        d.update(i)
    return d


def pipeline(file):
    data = get_patterns(file)
    result = set_intervals(data)
    return [dict_update(data) for data in result]

def split_time(label, data):
    return [int(x) for x in data[label].split('.')]

def set_hour(label, data):
    h, m, s = split_time(label, data)
    date = datetime.now()
    return date.replace(hour=h,minute=m,second=s,microsecond=0)

data = pipeline(f)

infos = []

for d in data:
    registration_start = set_hour('registration_start', d)
    registration_end = set_hour('registration_end', d)
    seizure_start = set_hour('seizure_start', d)
    seizure_end = set_hour('seizure_end', d)

    total = registration_end - registration_start
    seizure_end_sec =  registration_end - seizure_start 
    seizure_start_sec = registration_end - seizure_end

    d['total'] = int(total.total_seconds())
    d['seizure_start_sec'] = int(seizure_start_sec.total_seconds())
    d['seizure_end_sec'] = int(seizure_end_sec.total_seconds())
    d['total_seizure'] = d['seizure_end_sec'] - d['seizure_start_sec'] 

    infos.append(d)


## Infos de Sinais

In [75]:
from mne.io import read_raw_edf

def get_infos_edf(path:str, file:dict) -> dict:
    use_keys = ['meas_date','highpass','lowpass','sfreq','ch_names','nchan']
    sg = dict(read_raw_edf(f"{path}{file['name']}").info, verbose=False)
    result = {key:sg[key] for key in sg.keys() if key in use_keys}
    result['meas_date'] = result['meas_date'].strftime("%Y-%m-%d")
    return {**file,**result} 


In [76]:

path_data = '/mnt/c/Users/davi.nascimento/Documents/siena-eeg/PN00/'

result = [get_infos_edf(path_data,info) for info in infos]

Extracting EDF parameters from /mnt/c/Users/davi.nascimento/Documents/siena-eeg/PN00/PN00-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/c/Users/davi.nascimento/Documents/siena-eeg/PN00/PN00-2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/c/Users/davi.nascimento/Documents/siena-eeg/PN00/PN00-3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/c/Users/davi.nascimento/Documents/siena-eeg/PN00/PN00-4.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /mnt/c/Users/davi.nascimento/Documents/siena-eeg/PN00/PN00-5.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [77]:
import json

file = '/mnt/c/Users/davi.nascimento/Documents/github/doutorado_ppgee_v2/data/siena_infos.json'

with open(file, 'w', encoding='utf-8') as fl:
    json.dump(result, fl, ensure_ascii=False, indent=4)